In [1]:
import pandas as pd
import nltk
import nltk.tokenize as tok
import numpy as np
import sklearn as sk
import gensim as gen
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import string

# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')



In [2]:
data = pd.read_csv("news-train.csv")
test = pd.read_csv("test-full.csv")
print(data.keys())
print(test.keys())

Index(['Unnamed: 0', 'HEADER', 'MEDIANAME', 'CAT', 'WEBSITE', 'PTIME'], dtype='object')
Index(['Unnamed: 0', 'HEADER', 'MEDIANAME', 'WEBSITE', 'PTIME'], dtype='object')


In [3]:
HEADERS = np.append(data.HEADER.get_values(), test.HEADER.get_values())
HEADERS

array(["Charlie Sheen drunk at Taco Bell: 'Sorry I'm so f---ing hammered'",
       'First cars running Android Auto to roll out this year: Google',
       'GE Is Open to Alliance With France', ...,
       'Security Risks for Cloud Are Like Those for Non-Cloud Deployments',
       'AT&T, DirecTV Said to Aim to Complete Deal by Monday',
       'Climate change is here, government report says'], dtype=object)

In [4]:
mapl = lambda f, x: list(map(f, x))
filterl = lambda f, x: list(filter(f, x))
lm = WordNetLemmatizer()
stop_wrods = set(stopwords.words('english'))

# tokens = mapl(tok.wordpunct_tokenize, data.HEADER.get_values())
# tokens = mapl(lambda x: mapl(lm.lemmatize, x), tokens)
# tokens = mapl(lambda x: filterl(lambda y: not (y in stop_wrods or y in string.punctuation), x), tokens)
# print(tokens[0])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def tokenize(s):
    X = mapl(lm.lemmatize, tok.wordpunct_tokenize(s))
    return filterl(lambda y: not (y in stop_wrods or y in string.punctuation), X)

vectorizer = TfidfVectorizer(analyzer="word", tokenizer=tokenize)
vectorizer.fit(HEADERS)
vect_data = vectorizer.transform(data.HEADER)
vect_test = vectorizer.transform(test.HEADER)

In [6]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(n_estimators=50)
model = SVC(degree=2)
# model = KNeighborsClassifier(metric=cosine_similarity, n_jobs=3, n_neighbors=10)
params = {"C": range(1, 10), "kernel":["linear", "poly", "rbf", "sigmoid"]}
kfold = KFold(shuffle=True, n_splits=5)
vect_data[:10]

clf = GridSearchCV(param_grid=params, estimator=model, cv=kfold, n_jobs=3, scoring="accuracy")
clf.fit(vect_data[:1000], data.CAT[:1000])
# len(list(filter(lambda x: x, model.predict(vect_data[:100]) == data.CAT[:100])))
# vect_data.shape

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=3,
       param_grid={'C': range(1, 10), 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [9]:
clf.best_params_
model = SVC(verbose=True, C = 2, kernel="linear")
model.fit(vect_data, data.CAT)
res = model.predict(vect_test)
res
# model2 = SVC(degree = 2, C = 2, kernel="linear", verbose=True)
# model2.fit(vect_data[:1000], data.CAT[:1000])

[LibSVM]

array(['e', 'e', 'b', ..., 't', 'b', 't'], dtype=object)

In [12]:
open("out1.txt", "w").write("\n".join(res))

253451